In [1]:
import pandas as pd

# Import data from both CSV files (autodetect sep, skip malformed lines)
df_atual = pd.read_csv('./ServidoresMG_atual.csv', encoding='latin1', sep=None, engine='python', on_bad_lines='skip')
df_inicial = pd.read_csv('./ServidoresMG_inicial.csv', encoding='latin1', sep=None, engine='python', on_bad_lines='skip')

# Select only 'AUDITOR FISCAL DA RECEITA ESTADUAL' records
auditores_atual = df_atual[df_atual['nmefet'] == 'AUDITOR FISCAL DA RECEITA ESTADUAL']
auditores_inicial = df_inicial[df_inicial['nmefet'] == 'AUDITOR FISCAL DA RECEITA ESTADUAL']

In [6]:
# Import dados.csv
dados = pd.read_csv('./dados.csv', encoding='utf-8-sig', sep=None, engine='python', on_bad_lines='skip')

# Filter active auditors from auditores_inicial
auditores_ativos_inicial = auditores_inicial[auditores_inicial['descsitser'] == 'ATIVO']

# Find auditors in auditores_inicial that are not in dados (by MASP)
auditores_nao_em_dados = auditores_ativos_inicial[~auditores_ativos_inicial['masp'].isin(dados['MASP'])]

# Create new dataframe with all columns from dados.csv, filled with blanks
resultado = pd.DataFrame(columns=dados.columns)

# Fill in the specified columns
resultado['MASP'] = auditores_nao_em_dados['masp'].values
resultado['AREA'] = 'VETERANO'
resultado['NOME'] = auditores_nao_em_dados['nome'].values
resultado['SITUACAO'] = 'EM EXERCÍCIO'

# Save to CSV (use UTF-8 with BOM so Excel on Windows shows accents correctly and semicolon separator)
resultado.to_csv('./auditores_veteranos.csv', index=False, encoding='utf-8-sig', sep=';')
print(f"Arquivo criado com {len(resultado)} auditores.")

Arquivo criado com 0 auditores.


In [7]:
dados

,MASP,INSCRICAO,POSICAO_CONCURSO,AREA,NOME,PCD,SITUACAO,ORGAO_DESTINO,DATA_NOMEACAO,DATA_EXONERACAO,DATA_PUBLICACAO_EXONERACAO,DATA_NOMEACAO_SEM_EFEITO,DATA_INATIVIDADE,DATA_PUBLICACAO_INATIVIDADE,OBSERVACAO
0,7553860.0,266015163.0,1.0,FISCALIZAÇÃO,LUCIANA YUMI KAWASHIMA,NÃO,EM EXERCÍCIO,NaN,22/12/2023,NaN,NaN,NaN,NaN,NaN,NaN
1,7556178.0,266010881.0,2.0,FISCALIZAÇÃO,GIRLIAN ANTONIO DOS SANTOS,NÃO,EM EXERCÍCIO,NaN,22/12/2023,NaN,NaN,NaN,NaN,NaN,NaN
2,7553845.0,266001718.0,3.0,FISCALIZAÇÃO,DOUGLAS MOURA SIMOES PIMENTA,NÃO,EM EXERCÍCIO,NaN,22/12/2023,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,266002342.0,4.0,FISCALIZAÇÃO,EULER CAETANO SANTOS,NÃO,POSSE JUDICIAL,NaN,22/12/2023,NaN,NaN,NaN,NaN,NaN,Nomeação tornada sem efeito por ato no DOE do ...
4,7557614.0,266008016.0,5.0,FISCALIZAÇÃO,ROBERTO BLACK,NÃO,EM EXERCÍCIO,NaN,22/12/2023,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,2590255.0,NaN,NaN,VETERANO,WILTON ANTONIO VERCOSA,NaN,EM EXERCÍCIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3564,8927535.0,NaN,NaN,VETERANO,WILVER BERNARDES MENEGATTI,NaN,EM EXERCÍCIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3565,3869096.0,NaN,NaN,VETERANO,WLADIMIR CESAR DE ANDRADE,NaN,EM EXERCÍCIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3566,3711355.0,NaN,NaN,VETERANO,WLADIMIR ZLOCCOWICK MAIA,NaN,EM EXERCÍCIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Filter active auditors from dados.csv
auditores_exercicio_dados = dados[dados['SITUACAO'] == 'EM EXERCÍCIO']

# Filter active auditors from auditores_atual
auditores_ativos_atual = auditores_atual[auditores_atual['descsitser'] == 'ATIVO']

# Divergence 1: Auditors "EM EXERCÍCIO" in dados.csv but NOT in auditores_atual or INACTIVE
# Get MASPs from dados that are "EM EXERCÍCIO"
masps_exercicio_dados = set(auditores_exercicio_dados['MASP'].dropna().astype(int))

# Get MASPs from auditores_atual that are ATIVO
masps_ativos_atual = set(auditores_ativos_atual['masp'])

# Find MASPs in dados "EM EXERCÍCIO" but not in auditores_atual as ATIVO
divergencia_1 = masps_exercicio_dados - masps_ativos_atual

print(f"Divergência 1: {len(divergencia_1)} auditores constam como 'EM EXERCÍCIO' em dados.csv mas NÃO estão ATIVOS em auditores_atual")
print(f"MASPs: {sorted(divergencia_1)[:10]}..." if len(divergencia_1) > 10 else f"MASPs: {sorted(divergencia_1)}")

# Divergence 2: Auditors ATIVO in auditores_atual but NOT in dados.csv
divergencia_2 = masps_ativos_atual - masps_exercicio_dados

print(f"\nDivergência 2: {len(divergencia_2)} auditores estão ATIVOS em auditores_atual mas NÃO constam como 'EM EXERCÍCIO' em dados.csv")
print(f"MASPs: {sorted(divergencia_2)[:10]}..." if len(divergencia_2) > 10 else f"MASPs: {sorted(divergencia_2)}")

# Create detailed reports
if len(divergencia_1) > 0:
    relatorio_div1 = auditores_exercicio_dados[auditores_exercicio_dados['MASP'].isin(divergencia_1)][['MASP', 'NOME', 'SITUACAO']]
    print(f"\nDetalhes Divergência 1:")
    print(relatorio_div1.head(10))

if len(divergencia_2) > 0:
    relatorio_div2 = auditores_ativos_atual[auditores_ativos_atual['masp'].isin(divergencia_2)][['masp', 'nome', 'descsitser']]
    print(f"\nDetalhes Divergência 2:")
    print(relatorio_div2.head(10))

Divergência 1: 4 auditores constam como 'EM EXERCÍCIO' em dados.csv mas NÃO estão ATIVOS em auditores_atual
MASPs: [2849602, 3090636, 3869328, 6692669]

Divergência 2: 12 auditores estão ATIVOS em auditores_atual mas NÃO constam como 'EM EXERCÍCIO' em dados.csv
MASPs: [3181096, 6683411, 6691646, 7554124, 7554702, 7554983, 7555014, 7556095, 7556384, 7556400]...

Detalhes Divergência 1:
           MASP                   NOME      SITUACAO
2880  3869328.0  DAISI SOARES TEIXEIRA  EM EXERCÍCIO
2882  6692669.0  DANIEL CARVALHO LAIER  EM EXERCÍCIO
2942  2849602.0   ELI RIBEIRO FERREIRA  EM EXERCÍCIO
3356  3090636.0   PAULO FIALHO QUINTAO  EM EXERCÍCIO

Detalhes Divergência 2:
           masp                                          nome descsitser
17568   7555014                   ALEXANDER RODRIGUES LADEIRA      ATIVO
56721   6683411               ANTONIO RICARDO FERREIRA CALDAS      ATIVO
145653  7556400                           EDUARDO VERGNE DIAS      ATIVO
183618  7554124  FELIPE AUGUST

In [4]:
auditores_exercicio_dados

,MASP,INSCRICAO,POSICAO_CONCURSO,AREA,NOME,PCD,SITUACAO,ORGAO_DESTINO,DATA_NOMEACAO,DATA_EXONERACAO,DATA_PUBLICACAO_EXONERACAO,DATA_NOMEACAO_SEM_EFEITO,DATA_INATIVIDADE,DATA_PUBLICACAO_INATIVIDADE,OBSERVACAO
